In [1]:
#In this notebook, the code loops through a series of folders containing medical images
#in DICOM (.dcm) format,
#retrieves the images, processes them by resizing and normalizing them to 256x256 pixels, 
#and saves them as JPG files into an output folder.

In [2]:
import os
import shutil
import numpy as np
import pydicom
from PIL import Image
#path de mon fichier dans sataset
root_dir = "/kaggle/input/medicale-dataset"

# makdirs pour crier un dossier pour les images jpg 
output_root = "/kaggle/working/output_jpg/"
os.makedirs(output_root, exist_ok=True)

# Parcours des sous-dossiers jsq a au drn
for patient_folder in sorted(os.listdir(root_dir)):
    patient_path = os.path.join(root_dir, patient_folder)

    if not os.path.isdir(patient_path):
        continue

    print(f"Traitement du patient: {patient_folder}")

    try:
        # Recherche dicom  .dcm
        dicom_files = []
        for root, _, files in os.walk(patient_path):
            for file_name in sorted(files):
                if file_name.endswith(".dcm"):
                    file_path = os.path.join(root, file_name)
                    dicom_files.append(pydicom.dcmread(file_path))

        if not dicom_files:
            print(f"Aucun fichier DICOM dans {patient_folder}")
            continue

        # Tri par position Z
        dicom_files.sort(key=lambda x: float(x.ImagePositionPatient[2]))
        volume = np.stack([d.pixel_array for d in dicom_files])

        # Limite  128 slices=volume
        volume = volume[:128]

        #  dossier de sortie pour le patient
        patient_output = os.path.join(output_root, patient_folder)
        os.makedirs(patient_output, exist_ok=True)

        for i, slice_img in enumerate(volume):
            img = slice_img.astype("float32")
            img = (img - img.min()) / (img.max() - img.min() + 1e-5)
            img = (img * 255).astype("uint8")
            img_pil = Image.fromarray(img).resize((256, 256)) #resize 
            img_pil.save(os.path.join(patient_output, f"slice_{i:03d}.jpg"))

        print(f"Patient '{patient_folder}' traité avec {len(volume)} slices.")

    except Exception as e:
        print(f"Erreur dans '{patient_folder}': {e}")
        # I used DICOM dataset folders, converted them to .zip, 
        #and uploaded them to Kaggle datasets (it takes a little while to load). Then
        #in your notebook,upload the input and add your already uploaded dataset."
     


# Créer un fichier ZIP de tout le dossier d'images
shutil.make_archive("/kaggle/working/output_images", 'zip', output_root) #pour telecharger les images .zip

Traitement du patient: PANCREAS_0001
Patient 'PANCREAS_0001' traité avec 128 slices.
Traitement du patient: PANCREAS_0002
Patient 'PANCREAS_0002' traité avec 128 slices.
Traitement du patient: PANCREAS_0003
Patient 'PANCREAS_0003' traité avec 128 slices.
Traitement du patient: Pancreas-CT
Patient 'Pancreas-CT' traité avec 128 slices.
Traitement du patient: Pancreas-CT_3
Patient 'Pancreas-CT_3' traité avec 128 slices.
Traitement du patient: Pancreas-CT_4
Patient 'Pancreas-CT_4' traité avec 128 slices.
Traitement du patient: Pancreas-CT_5
Patient 'Pancreas-CT_5' traité avec 128 slices.
Traitement du patient: Pancreas-CT_6
Patient 'Pancreas-CT_6' traité avec 128 slices.
Traitement du patient: Pancreas-CT_7
Patient 'Pancreas-CT_7' traité avec 128 slices.


'/kaggle/working/output_images.zip'